In [35]:
from numpy import random
import numpy as np
import itertools 
fL=1.4
f1=1.4
d=3
landaL=10**-6
lookup={}

In [36]:
def Calculate_landa(landaL,d,f1,fL):
    landa=(landaL)*(10**((d*(1-fL))/(1-f1)))
    return landa
    

In [37]:
def Reliability(landa,c,f):
    r=np.exp(-landa*(c/f))
    return r

In [38]:
def Create_Tasks(num,l):
    Set_length=num
    w, h = 3, Set_length;
    Matrix = [[0 for x in range(w)] for y in range(h)]
    x = random.uniform(0.75,1.25,size=(Set_length))
    x=np.sort(x)
    for i in range(Set_length):
        Matrix[i][0]=x[i]
        Matrix[i][1]=Set_length-i
        Matrix[i][2]=l #1.4GHz
        lookup["T"+str(i+1)]=Matrix[i]    
        
    return lookup

In [39]:
def getList(dict): 
    list= [] 
    for key in dict.keys(): 
        list.append(key)
          
    return list

In [40]:
def findsubsets(s,n): 
    return list(itertools.combinations(s,n)) 

In [41]:
sum1=0
for k in range(100):
    lookup={}
    Tasks=Create_Tasks(5,fL)

    set1={0}
    keys=getList(lookup)
    SubSet=[]
    for j in range (len(keys)):
        SubSet.append(keys[0:j+1])
    SubSet.remove(SubSet[len(SubSet)-1])

    TotalSub=[]
    for i in range (len(SubSet)):
        sub=[]
        for j in range(len(SubSet[i])):
            find=findsubsets(SubSet[i],j+1)
            sub.append(find)
        TotalSub.append(sub)


    t=[]
    for i in range (len(TotalSub)):
        e=[]
        for j in range(len(TotalSub[i])):
            for k in range (len(TotalSub[i][j])):
                e.append(TotalSub[i][j][k])
        t.append(e)        

    fail=[]
    for i in range(len(t)):
        q=[]
        for j in range(len(t[i])):
            sum=0
            for k in range (len(t[i][j])):
                sum=sum+lookup[t[i][j][k]][0]
            if(sum+lookup[keys[i+1]][0]<3):
                q.append(t[i][j])

        fail.append(q)


    Success=[]

    for i in range(len(fail)):
        temp=[]
        for j in range(len(fail[i])):
            main_list = np.setdiff1d(SubSet[i],fail[i][j])
            temp.append(list(main_list))
        Success.append(temp)


    mulfail=1
    mulsuc=1
    prob=0
    reli1=0
    reli=0
    Prob_set=[]
    for i in range(len(fail)):
        prob=0
        for j in range(len(fail[i])):
            for k in range(len(fail[i][j])):
                lan=Calculate_landa(landaL,d,f1,fL)
                task=lookup[fail[i][j][k]]
                reli1=Reliability(lan,task[0],task[2])
                #print(reli)
                mulfail=mulfail*(1-reli1)

            for p in range(len(Success[i][j])):
                lan=Calculate_landa(landaL,d,f1,fL)
                task=lookup[Success[i][j][p]]
                reli=Reliability(lan,task[0],task[2])
                mulsuc=mulsuc*(reli)
            
            prob=prob+mulfail*mulsuc
            mulfail=1
            mulsuc=1
        Prob_set.append(prob)

    Reliability_Schedule=[0]
    for i in range(len(Prob_set)):
        lan=Calculate_landa(landaL,d,f1,fL)
        task=lookup[keys[i+1]]
        reli2=Reliability(lan,task[0],task[2])
        Reliability_Schedule.append(Prob_set[i]*reli2)


    Reliability_Task_i=[]
    for i in range(len(keys)):
        lan=Calculate_landa(landaL,d,f1,fL)
        task=lookup[keys[i]]
        reli3=Reliability(lan,task[0],task[2])
        Reliability_Task_i.append(reli3)


    Reliability_System=1
    for i in range(len(Reliability_Schedule)):
        Reliability_System=Reliability_System*(1-((1-Reliability_Task_i[i])*(1-Reliability_Schedule[i])))
    print("=======>>>>>>Tasks:")
    print(Tasks)
    print("=======>>>>>>Groups obtained based on failed tasks and default slack :")
    for i in range(len(fail)):
        for j in range(len(fail[i])):
            for k in range(i+2):
                if(len(fail[i][j])==k):
                    print("G"+str(i+2)+","+str(k)+":"+str(fail[i][j]))
    print("=======>>>>>>Prob(P(i,j)):")
    print(Prob_set)
    print("=======>>>>>>R_i^s(S):")
    print(Reliability_Schedule)
    print("=======>>>>>>R_i^t:")
    print(Reliability_Task_i)
    print("=======>>>>>>R_sys(S):")
    print(Reliability_System)
    print("=======>>>>>>Probability of failure:")
    print(1-Reliability_System)
    print("-------------------------------------------------------------------------------")
    sum1=sum1+(1-Reliability_System)

print("Avrage Probability of failure:",sum1/100)
    
    


=======>>>>>>Tasks:
{'T1': [0.8881407856976764, 5, 1.4], 'T2': [0.889820785959998, 4, 1.4], 'T3': [0.9717475223944054, 3, 1.4], 'T4': [1.0246518810309182, 2, 1.4], 'T5': [1.236915429666098, 1, 1.4]}
=======>>>>>>Groups obtained based on failed tasks and default slack :
G2,1:('T1',)
G3,1:('T1',)
G3,1:('T2',)
G3,2:('T1', 'T2')
G4,1:('T1',)
G4,1:('T2',)
G4,1:('T3',)
G4,2:('T1', 'T2')
G4,2:('T1', 'T3')
G4,2:('T2', 'T3')
G5,1:('T1',)
G5,1:('T2',)
G5,1:('T3',)
G5,1:('T4',)
=======>>>>>>Prob(P(i,j)):
[0.0006341850950694061, 0.0012691664773104047, 0.001962150105834963, 0.0026896233323711704]
=======>>>>>>R_i^s(S):
[0, 0.0006337821437954503, 0.0012682858476993242, 0.0019607145449549515, 0.0026872480699539417]
=======>>>>>>R_i^t:
[0.9993658149049306, 0.999364615666485, 0.9993061354622708, 0.999268373568494, 0.999116879159754]
=======>>>>>>R_sys(S):
0.9964319917766474
=======>>>>>>Probability of failure:
0.00356800822335257
-------------------------------------------------------------------------

G5,1:('T4',)
G5,2:('T1', 'T2')
G5,2:('T1', 'T3')
G5,2:('T1', 'T4')
G5,2:('T2', 'T3')
G5,2:('T2', 'T4')
G5,2:('T3', 'T4')
=======>>>>>>Prob(P(i,j)):
[0.0005580881592381903, 0.0011245474025357242, 0.0017539301673278837, 0.0023874882061482748]
=======>>>>>>R_i^s(S):
[0, 0.0005577718485125443, 0.0011238388347401748, 0.0017528169972075458, 0.0023857216046166865]
=======>>>>>>R_i^t:
[0.9994419118407618, 0.9994332244459768, 0.9993699084681076, 0.9993653281407242, 0.9992600585305347]
=======>>>>>>R_sys(S):
0.9968782285277081
=======>>>>>>Probability of failure:
0.0031217714722918677
-------------------------------------------------------------------------------
=======>>>>>>Tasks:
{'T1': [0.7664284434272888, 5, 1.4], 'T2': [0.7937415905526612, 4, 1.4], 'T3': [1.0144147365134861, 3, 1.4], 'T4': [1.2013091459494978, 2, 1.4], 'T5': [1.2333656394605872, 1, 1.4]}
=======>>>>>>Groups obtained based on failed tasks and default slack :
G2,1:('T1',)
G3,1:('T1',)
G3,1:('T2',)
G3,2:('T1', 'T2')
G4,1:('T1

=======>>>>>>Tasks:
{'T1': [0.8261363427379497, 5, 1.4], 'T2': [0.8534075155078155, 4, 1.4], 'T3': [0.9329312417445238, 3, 1.4], 'T4': [0.9526552453861922, 2, 1.4], 'T5': [1.0142036682815918, 1, 1.4]}
=======>>>>>>Groups obtained based on failed tasks and default slack :
G2,1:('T1',)
G3,1:('T1',)
G3,1:('T2',)
G3,2:('T1', 'T2')
G4,1:('T1',)
G4,1:('T2',)
G4,1:('T3',)
G4,2:('T1', 'T2')
G4,2:('T1', 'T3')
G4,2:('T2', 'T3')
G5,1:('T1',)
G5,1:('T2',)
G5,1:('T3',)
G5,1:('T4',)
G5,2:('T1', 'T2')
G5,2:('T1', 'T3')
G5,2:('T1', 'T4')
G5,2:('T2', 'T3')
G5,2:('T2', 'T4')
G5,2:('T3', 'T4')
=======>>>>>>Prob(P(i,j)):
[0.0005899233144481819, 0.001198954863065917, 0.0018643134077481517, 0.0025432810124704328]
=======>>>>>>R_i^s(S):
[0, 0.0005895638204645355, 0.00119815617031941, 0.0018630452335966887, 0.002541439247572436]
=======>>>>>>R_i^t:
[0.9994100766855518, 0.9993906089573986, 0.9993338425230917, 0.9993197634334482, 0.9992758311452938]
=======>>>>>>R_sys(S):
0.9967386450654239
=======>>>>>>Probabi

=======>>>>>>Tasks:
{'T1': [0.7586674221380897, 5, 1.4], 'T2': [0.9744310775943001, 4, 1.4], 'T3': [1.079338426824974, 3, 1.4], 'T4': [1.108192278819728, 2, 1.4], 'T5': [1.1436651133496234, 1, 1.4]}
=======>>>>>>Groups obtained based on failed tasks and default slack :
G2,1:('T1',)
G3,1:('T1',)
G3,1:('T2',)
G3,2:('T1', 'T2')
G4,1:('T1',)
G4,1:('T2',)
G4,1:('T3',)
G4,2:('T1', 'T2')
G4,2:('T1', 'T3')
G5,1:('T1',)
G5,1:('T2',)
G5,1:('T3',)
G5,1:('T4',)
G5,2:('T1', 'T2')
G5,2:('T1', 'T3')
=======>>>>>>Prob(P(i,j)):
[0.000541758497368483, 0.001237161583643601, 0.002006330853894628, 0.0027944081079674587]
=======>>>>>>R_i^s(S):
[0, 0.000541381552624379, 0.0012362081540473841, 0.0020047433391737583, 0.0027921262779223286]
=======>>>>>>R_i^t:
[0.9994582415026315, 0.9993042199689808, 0.9992293410910732, 0.9992087472922085, 0.999183429922557]
=======>>>>>>R_sys(S):
0.9963943626079654
=======>>>>>>Probability of failure:
0.0036056373920345663
------------------------------------------------------

G5,2:('T1', 'T2')
G5,2:('T1', 'T3')
G5,2:('T1', 'T4')
G5,2:('T2', 'T3')
=======>>>>>>Prob(P(i,j)):
[0.0005478017938638224, 0.001133083879612237, 0.0017673269386189311, 0.0024630219469571977]
=======>>>>>>R_i^s(S):
[0, 0.0005474809995556356, 0.0011323644135801572, 0.0017660937338648882, 0.0024608642228050124]
=======>>>>>>R_i^t:
[0.9994521982061362, 0.9994143971199434, 0.9993650372712688, 0.9993022203605368, 0.9991239525271585]
=======>>>>>>R_sys(S):
0.9966666538258871
=======>>>>>>Probability of failure:
0.00333334617411285
-------------------------------------------------------------------------------
=======>>>>>>Tasks:
{'T1': [0.9571322534470061, 5, 1.4], 'T2': [1.051743698834653, 4, 1.4], 'T3': [1.0760813691861093, 3, 1.4], 'T4': [1.1219719473529177, 2, 1.4], 'T5': [1.1992281068217214, 1, 1.4]}
=======>>>>>>Groups obtained based on failed tasks and default slack :
G2,1:('T1',)
G3,1:('T1',)
G3,1:('T2',)
G4,1:('T1',)
G4,1:('T2',)
G4,1:('T3',)
G5,1:('T1',)
G5,1:('T2',)
G5,1:('T3',)
G5